In [18]:
import os
import glob
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import warnings
from tqdm import tqdm_notebook as tqdm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import utils
from constants import DATA_DIR
warnings.filterwarnings('ignore')

In [63]:
weights = ['base','sqrt','linear','quad']
seasons = ['2016-17','2017-18','2018-19','2019-20']

for weight in weights:
    df = pd.DataFrame()
    mean = []
    for season in seasons:
        df1 = pd.read_csv(os.path.join(DATA_DIR,'Weighted',weight,season+'.csv'))
        df = df.append(df1)
    df.fillna(0, inplace=True)
    X = df.loc[:, df.columns != 'VALUE']._get_numeric_data()
    y = df['VALUE']
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=None)
    cv = RepeatedKFold(n_splits=2, n_repeats=1000)
    
    reg = LinearRegression()
    scores = cross_val_score(reg, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv)
    model = reg.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mean.append(np.mean(scores))
    print(weight)
    print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
    print(mae(y_test, y_pred))
    print(mse(y_test, y_pred, squared=False))
    print(r2(y_test, y_pred))
    print('\n')
    
    #print(sum(mean) / len(mean))

base
Accuracy: -2.032 (0.060)
2.101902874253372
2.7309948663894716
0.5421193442301511


sqrt
Accuracy: -2.025 (0.060)
2.011288108656903
2.710368043069314
0.5033347249459064


linear
Accuracy: -1.983 (0.060)
2.019474849804771
2.6380193327973394
0.6041351183932735


quad
Accuracy: -1.851 (0.059)
2.25911357193135
2.913820840097076
0.533433477006312


